# Paper tutorial

In [ ]:
import pybinding as pb
import matplotlib.pyplot as plt
import numpy as np
import scipy

from matplotlib import gridspec

pb.pltutils.use_style()

## Graphene lattice

In [ ]:
def monolayer_graphene():
    from math import sqrt
    a = 0.24595  # [nm] unit cell length
    a_cc = 0.142 # [nm] carbon-carbon distance
    t = -2.8     # [eV] nearest neighbour pz-pz hopping
    # create a lattice with 2 primitive vectors
    lat = pb.Lattice(
        a1=[a, 0],
        a2=[a/2, a/2*sqrt(3)])
    # add orbitals 'A' and 'B' (sublattices)
    lat.add_sublattices(
        ('A', [0, -a_cc/2]),
        ('B', [0,  a_cc/2]))
    # add hoppings
    lat.add_hoppings(
        # inside the main cell
        ([0,  0], 'A', 'B', t),
        # between neighbouring cells
        ([1, -1], 'A', 'B', t),
        ([0, -1], 'A', 'B', t))
    return lat

In [ ]:
fig = plt.figure(figsize=(3, 3))
lat = monolayer_graphene()
lat.plot()

In [ ]:
fig = plt.figure(figsize=(3, 3))
lat.plot_brillouin_zone()
plt.xlim([-30, 30])
plt.xticks([-20, 0, 20])

In [ ]:
fig = plt.figure(figsize=(3, 3))
model = pb.Model(lat, pb.translational_symmetry(a1=True, a2=True))
model.plot()

In [ ]:
# Simple shapes
rectangle = pb.rectangle(x=6, y=1)
hexagon = pb.regular_polygon(num_sides=6, radius=1.92, angle=np.pi/6)
circle = pb.circle(radius=0.6)

# Compose them naturally
shape = rectangle + hexagon - circle

fig = plt.figure(figsize=(3, 3))
model = pb.Model(lat, shape)
model.shape.plot()
model.plot()
plt.xlim([-3,3])
plt.xticks([-2, 0, 2])

In [ ]:
from pybinding.repository import graphene
model = pb.Model(
    graphene.monolayer_4atom(),
    pb.rectangle(x=2, y=2),
    pb.translational_symmetry(a1=1.2, a2=False)
)

fig = plt.figure(figsize=(3, 3))
model.plot()

# Potential

In [ ]:
@pb.onsite_energy_modifier
def potential(x, y):
    return np.sin(x)**2 + np.cos(y)**2

from pybinding.constants import phi0
from math import pi
def constant_magnetic_field(B):
    @pb.hopping_energy_modifier
    def function(energy, x1, y1, x2, y2):
        # the midpoint between two sites
        y = 0.5 * (y1 + y2)
        # scale from nanometers to meters
        y *= 1e-9

        # vector potential along the x-axis
        A_x = B * y

        # integral of (A * dl) from position 1 to position 2
        peierls = A_x * (x1 - x2)
        # scale from nanometers to meters (because of x1 and x2)
        peierls *= 1e-9

        # the Peierls substitution
        return energy * np.exp(1j * 2*pi/phi0 * peierls)
    return function

In [ ]:
from pybinding.repository import graphene

model = pb.Model(
    graphene.monolayer(),
    pb.rectangle(12),
    potential
)

fig = plt.figure(figsize=(3, 3))
model.onsite_map.plot_contourf()

In [ ]:
import random
def random_vacancies(concentration):
    """ Add random vacancies to the sample based on the concentration parameter
        every time script is ran a different configuration will be defined
    """

    @pb.site_state_modifier(min_neighbors=2)  # limit the number of neighbours for each site, removes dangling sites
    def modifier(state):
        # total number of states
        total_num = len(state)
        # number of vacancies based on the concentration
        vacanc_num = int(concentration / 100 * total_num)

        # seed random number generator with randint
        np.random.seed(random.randint(0, 2 ** 32 - 1))
        # generate vacanc_num integer values that would specify vacant sites, [0, total_num - 1]
        vacant_sites = np.random.randint(0, total_num, vacanc_num)
        # remove these sites
        state[vacant_sites] = False
        return state

    return modifier

model = pb.Model(
    graphene.monolayer(),
    pb.rectangle(6),
    random_vacancies(1)
)

In [ ]:
fig = plt.figure(figsize=(3, 3))
model.plot()
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.xticks([-2, 0, 2])

In [ ]:
def triaxial_displacement(c):
    @pb.site_position_modifier
    def displacement(x, y, z):
        ux = 2*c * x*y
        uy = c * (x**2 - y**2)
        return x + ux, y + uy, z
    return displacement

model = pb.Model(
    graphene.monolayer(),
    pb.rectangle(6),
    triaxial_displacement(0.04)
)

In [ ]:
fig = plt.figure(figsize=(3, 3))
model.plot()
plt.xlim([-4, 4])
plt.ylim([-4, 4])
plt.xticks([-2, 0, 2])
plt.yticks([-2, 0, 2])

# Borophene


In [ ]:
model = pb.Model(
    graphene.monolayer(),
    pb.rectangle(6)
)

fig = plt.figure(figsize=(3, 3))
solver = pb.solver.arpack(model, k=20)  # for the 20 lowest energy eigenvalues
eigenvalues = solver.calc_eigenvalues()
eigenvalues.plot()

In [ ]:
fig = plt.figure(figsize=(3, 3))
ldos_map = solver.calc_spatial_ldos(energy=0, broadening=0.05)  # [eV]
ldos_map.plot()
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.yticks([-2, 0, 2])
plt.xticks([-2, 0, 2])